In [7]:
!pip install pymongo faker

  Using cached pymongo-4.15.2-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (22 kB)
  Using cached faker-37.8.0-py3-none-any.whl.metadata (15 kB)
  Using cached dnspython-2.8.0-py3-none-any.whl.metadata (5.7 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pymongo-4.15.2-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl (1.7 MB)
Using cached dnspython-2.8.0-py3-none-any.whl (331 kB)
Using cached faker-37.8.0-py3-none-any.whl (2.0 MB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [faker]32m3/4 [faker]o]n]


In [12]:
from pymongo import MongoClient
from faker import Faker
import random
from datetime import datetime, timedelta 
import pandas as pd
from bson import ObjectId



Task 1.1: Create Database and Collections

In [13]:
client = MongoClient("mongodb://localhost:27017/")
db = client["EduHub"]

users_collection = db["users"]
courses_collection = db["courses"]
lessons_collection = db["lessons"]
enrollments_collection = db["enrollments"]
assignments_collection = db["assignments"]
submissions_collection = db["submissions"]

Task 1.2: Design Document Schemas

In [14]:
# user data 
user = [
    {
        "_id": ObjectId (),
        "userId": "string (unique)",
        "email": "string (unique, required)",
        "firstName": "string (required)",
        "lastName": "string (required)",
        "role": "(enum: ['student', 'instructor'])",
        "dateJoined": "datetime",
        "profile": {
            "bio": "string",
            "avatar": "string",
            "skills": ["string"]
        },
        "is_active": True
    }
]

Part 2: Data Population

In [15]:
#user data schema 
fake = Faker()

# Adding students and instructors programmatically

firstname = ["Adams", "John", "Cooper", "Doe", "Jane", "Smith", "Emily", "Johnson", "Oladele", "Michael", "Brown", "Sarah", "Williams", "David", "Adebayo", "Bamidele","Jones", "Linda", "Miller", "James"]

lastname = ["Ruth", "Alice", "Davis", "Robert", "Mary", "Grace", "Patricia", "Martinez", "Jennifer", "Pelumi", "Elizabeth", "Betrice", "Barbara", "John", "Susan", "Wilson","Jessica", "Anderson", "Sarah", "Thomas"]



skills = ["mathematics", "programming", "python", "sql", "excel", "analysis"]

users = []

for i in range(20):  # Adding 15 sample students and 5 instructors 
    user = {
        "_id" : ObjectId(), 
        "userId": f"user_{i + 1}",
        "firstname": random.choice(firstname),
        "lastname": random.choice(lastname),
        "role": "student" if i < 15 else "instructor",
        "dateJoined": datetime.now() - timedelta(days=random.randint(0, 500)),
        "profile": {
            "bio": fake.text(max_nb_chars=200),
            "avatar": fake.image_url(),
            "skills": random.sample(skills, random.randint(1, 5))
        },
        "is_active": random.choice([True, True, False])
    }
    email = f"{user['firstname'].lower()}.{user['lastname'].lower()}@eduhub.com" if user["role"] == "student" \
        else f"{user['firstname'].lower()}.{user['lastname'].lower()}@instructor.eduhub.com"
    user["email"] = email
    users.append(user)

#Insert users in users document collection 

def insert_many_documents(collection, documents):
    try:
        result = collection.insert_many(documents)
        return result.inserted_ids
    except Exception as e:
        print(f"An error occurred: {e}")
        return []

result = insert_many_documents(users_collection, users)
print(f"Inserted {len(result)} documents")

Inserted 20 documents


In [16]:
# Course Data
course = [
    {"title": "Python", "category": "Programming", "tags": ["Python", "Programming"], "courseId": "PYT"},
    {"title": "MongoDB", "category": "Database", "tags": ["Python", "Database"], "courseId": "EXL"},
    {"title": "JavaScript", "category": "Programming", "tags": ["JavaScript", "Programming"], "courseId": "JSC"},
    {"title": "Data Science", "category": "Data", "tags": ["Python", "Data"], "courseId": "DSI"},
    {"title": "Data Engineering", "category": "Data", "tags": ["Python", "Programming"], "courseId": "DEG"},
    {"title": "Data Visualization", "category": "Data", "tags": ["Python", "Programming"], "courseId": "DVI"},
    {"title": "SQL","category": "Database", "tags": ["SQL", "Database"], "courseId": "SQL"},
    {"title": "Data Analysis", "category": "Data", "tags": ["Excel", "Analysis"], "courseId": "DAS"}
]



instructor = [u for u in users if u['role'] == 'instructor']

courses = []


for item in course:

    instructorId = random.choice(instructor)["userId"]

    course_data = {
        "_id": ObjectId(),
        "courseId": item["courseId"],
        "title": item["title"],
        "description": f"Learn {item['title']} from scratch",
        "instructorId": instructorId,
        "category": item["category"],
        "level": random.choice(["beginner", "intermediate", "advanced"]),
        "duration": random.randint(5, 10) * 7 * 24, # duration in hours
        "tags": item["tags"],
        "price": random.randint(50, 300),
        "createdAt": datetime.now(),
        "updatedAt": datetime.now(),
        "isPublished": random.choice([True, False])
    }
    courses.append(course_data)


result = insert_many_documents(courses_collection, courses)
print(f"Inserted {len(result)} documents into course_collection")

Inserted 8 documents into course_collection


In [17]:
# lessons data
lessons = []

instructors = [u for u in users if u['role'] == 'instructor']

while len(lessons) < 25:

    instructorId = random.choice(instructors)["userId"]

    course = courses_collection.find_one({"instructorId": instructorId})

    if not course:
        continue
        
    lesson_data = {
        "_id": ObjectId(),
        "lessonId": f"lesson_{random.randint(100, 500)}",
        "chapter": f"Chapter {random.randint(1, 20)}",
        "description": f"Lesson on {course['title']}",
        "courseId": course["courseId"],
        "instructorId": instructorId,
        "duration": course['duration'] // (random.randint(1, 3) * 7 * 24) if course['duration'] > 0 else 1,
        "createdAt": datetime.now(),
        "updatedAt": datetime.now()
    }
    lessons.append(lesson_data)


result = insert_many_documents(lessons_collection, lessons)
print(f"Inserted {len(result)} documents into lessons_collection")

Inserted 25 documents into lessons_collection


In [18]:
# Enrollments collection creation 
students = [u for u in users if u['role'] == 'student']

enrollments = []

for i in range(15):

    course = courses_collection.find_one({"instructorId": instructorId})

    enrollment_data = {
        "_id": ObjectId(),
        "enrollmentId": f"{i:04d}",
        "courseId": random.choice(courses)["courseId"],
        "studentId": random.choice(students)["userId"],
        "startDate": datetime.now(),
        "endDate": datetime.now() + timedelta(hours=course["duration"])
    }
    enrollments.append(enrollment_data)

result = insert_many_documents(enrollments_collection, enrollments)
print(f"Inserted {len(result)} documents into enrollments_collection")

Inserted 15 documents into enrollments_collection


In [19]:
# Assignments 

assignments = []

for i in range(10):

    assignment_data = {
        "_id": ObjectId(),
        "assignmentId": f"a_{i:04d}",
        "dueDate": datetime.now() + timedelta(days=random.randint(1, 20)),
        "lessonId": random.choice(lessons)["lessonId"],
        "courseId": random.choice(courses)["courseId"],
        "description": f"Complete the assignment for {random.choice(courses)['title']}",
        "grades": {student["userId"]: random.randint(50, 100) for student in students}
    }
    assignments.append(assignment_data)



result = insert_many_documents(assignments_collection, assignments)
print(f"Inserted {len(result)} documents into assignments_collection")


Inserted 10 documents into assignments_collection


In [20]:
# Submissions 12

submissions = []

for i in range(12):
    
    submission_data = {
        "_Id": ObjectId(),
        "assignmentId": random.choice(assignments)["assignmentId"],
        "studentId": random.choice(students)["userId"],
        "description": f"Submission for {random.choice(assignments)['assignmentId']}",
        "submissionDate": datetime.now() - timedelta(days=random.randint(0, 10)),
        "isSubmitted": random.choice([True, False])
    }
    submissions.append(submission_data)

result = insert_many_documents(submissions_collection, submissions)
print(f"Inserted {len(result)} documents into submissions_collection")

Inserted 12 documents into submissions_collection


Part 3: Basic CRUD Operations (25 points)
Task 3.1: Create Operations

Write Python code using PyMongo to:

    Add a new student user
    Create a new course
    Enroll a student in a course
    Add a new lesson to an existing course


In [21]:
#Add a new student 

user = {
        "_id" : ObjectId(), 
        "userId": f"user_{users_collection.count_documents({}) + 1}",
        "firstname": "Mayowa",
        "lastname": "Williams",
        "role": "student",
        "dateJoined": datetime.now(),
        "profile": {
            "bio": fake.text(max_nb_chars=200),
            "avatar": fake.image_url(),
            "skills": random.sample(skills, random.randint(1, 3))
        },
        "is_active": True
    }
email = f"{user['firstname'].lower()}.{user['lastname'].lower()}@example.com"
user["email"] = email
    

def insert_one(collection, document):
    try:
        result = collection.insert_one(document)
        return result.inserted_id
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

result = insert_one(users_collection, user)
print(f"Inserted {result} document")

Inserted 68de2b01de87aa09a1f6bafa document


In [22]:
#Create a new course 
course = [
    {"title": "Go", "category": "Programming", "tags": ["Go", "Programming"], "courseId": "GOT"},
]

instructor = [u for u in users if u['role'] == 'instructor']


for item in course:

    instructorId = random.choice(instructor)["userId"]

    course_data = {
        "_id": ObjectId(),
        "courseId": item["courseId"],
        "title": item["title"],
        "description": f"Learn {item['title']} from scratch",
        "instructorId": instructorId,
        "category": item["category"],
        "level": random.choice(["beginner", "intermediate", "advanced"]),
        "duration": random.randint(2, 5) * 7 * 24, # duration in hours
        "tags": item["tags"],
        "price": random.randint(50, 300),
        "createdAt": datetime.now(),
        "updatedAt": datetime.now(),
        "isPublished": random.choice([True, False])
    }


result = insert_one(courses_collection, course_data)
print(f"Inserted {result} document into course_collection")

Inserted 68de2b02de87aa09a1f6bafb document into course_collection


In [23]:
#Enroll the new student in an existing course
new_student = users_collection.find_one({"firstname": "Mayowa", "lastname": "Williams"})
existing_course = courses_collection.find_one({"title": "Python"}) 
 
if new_student and existing_course:
    enrollment_data = {
        "_id": ObjectId(),
        "enrollmentId": f"{enrollments_collection.count_documents({}) + 1:04d}",
        "courseId": existing_course["courseId"],
        "studentId": new_student["userId"],
        "startDate": datetime.now(),
        "endDate": datetime.now() + timedelta(hours=existing_course["duration"])
    }
    
result = insert_one(enrollments_collection, enrollment_data)
print(f"Inserted {result} document into enrollments_collection")

Inserted 68de2b03de87aa09a1f6bafc document into enrollments_collection


In [24]:
#Add a new lesson to an existing course
course = courses_collection.find_one({"title": "Go"})

if course:
    lesson_data = {
        "_id": ObjectId(),
        "lessonId": f"lesson_{lessons_collection.count_documents({}) + 1}",
        "courseId": course["courseId"],
        "title": "Introduction to Go",
        "content": "This is the content of the lesson.",
        "duration": course['duration'] // (random.randint(1, 2) * 7 * 24) if course['duration'] > 0 else 1,
        "createdAt": datetime.now(),
        "updatedAt": datetime.now()
    }

result = insert_one(lessons_collection, lesson_data)
print(f"Inserted {result} document into lessons_collection")

Inserted 68de2b04de87aa09a1f6bafd document into lessons_collection


Task 3.2: Read Operations

Write Python queries to:

    Find all active students
    Retrieve course details with instructor information
    Get all courses in a specific category
    Find students enrolled in a particular course
    Search courses by title (case-insensitive, partial match)



In [25]:
#find all active students
from pymongo import MongoClient
client = MongoClient("mongodb://localhost:27017/")
db = client["EduHub"]
users_collection =  db["users"]

active_students = users_collection.find({"role": "student", "is_active": True})

for i in active_students:
    print(i)


{'_id': ObjectId('68de2afade87aa09a1f6ba96'), 'userId': 'user_3', 'firstname': 'Miller', 'lastname': 'Grace', 'role': 'student', 'dateJoined': datetime.datetime(2024, 6, 25, 8, 34, 18, 828000), 'profile': {'bio': 'Society them central word. More student fine require artist.\nWho six news. Rest kind relationship teacher technology consumer.', 'avatar': 'https://picsum.photos/200/76', 'skills': ['programming', 'sql']}, 'is_active': True, 'email': 'miller.grace@eduhub.com'}
{'_id': ObjectId('68de2afade87aa09a1f6ba98'), 'userId': 'user_5', 'firstname': 'Linda', 'lastname': 'Robert', 'role': 'student', 'dateJoined': datetime.datetime(2025, 1, 15, 8, 34, 18, 832000), 'profile': {'bio': 'Heavy skill ok again feel. Leave cause shoulder must wish evening. Trade yeah red fund small animal investment increase.', 'avatar': 'https://placekitten.com/17/440', 'skills': ['analysis', 'python', 'programming', 'sql']}, 'is_active': True, 'email': 'linda.robert@eduhub.com'}
{'_id': ObjectId('68de2afade87a

In [26]:
#List all courses taught by a specific instructor

# connect the instructorid to the users_collection to 
instructors = list(users_collection.find({"role": "instructor"}))

#   get the instructor details and  match the instructorId with the courses_collection to get the courses taught by the instructor

for instructor in instructors:
    instructorId = instructor["userId"]
    courses_taught = list(courses_collection.find({"instructorId": instructorId}))
    print(f"Instructor: {instructor['firstname']} {instructor['lastname']}")
    for course in courses_taught:
        print(f" - Course: {course['title']} (ID: {course['courseId']})")
    print("\n")


Instructor: Cooper Jennifer
 - Course: Data Science (ID: DSI)


Instructor: Doe Thomas
 - Course: Data Engineering (ID: DEG)


Instructor: James Robert
 - Course: Python (ID: PYT)
 - Course: SQL (ID: SQL)


Instructor: James Jessica
 - Course: Data Analysis (ID: DAS)


Instructor: Doe Davis
 - Course: MongoDB (ID: EXL)
 - Course: JavaScript (ID: JSC)
 - Course: Data Visualization (ID: DVI)
 - Course: Go (ID: GOT)




In [27]:
#get all courses in a specific category
category = "Data"
category_courses = list(courses_collection.find({"category": category}))

for course in category_courses:
    print(f"Course: {course['title']} (ID: {course['courseId']}) - Category: {course['category']}")

Course: Data Science (ID: DSI) - Category: Data
Course: Data Engineering (ID: DEG) - Category: Data
Course: Data Visualization (ID: DVI) - Category: Data
Course: Data Analysis (ID: DAS) - Category: Data


In [28]:
#find student enrolled in a particular course 

# Find the Python course 
course = courses_collection.find_one({"title": "Python"})
if course:
    # Get all enrollments for the course
    enrollments = list(enrollments_collection.find({"courseId": course["courseId"]}))

    # Step 3: Print results
    for enrollment in enrollments:
        student = users_collection.find_one({"userId": enrollment["studentId"]})
        if student:
            print(f"Student: {student['firstname']} {student['lastname']} "
                  f"is enrolled in Course: {course['title']} (ID: {course['courseId']})")


Student: Sarah Alice is enrolled in Course: Python (ID: PYT)
Student: Adams Susan is enrolled in Course: Python (ID: PYT)
Student: Mayowa Williams is enrolled in Course: Python (ID: PYT)


In [29]:
#search courses by title (case insensitive, partial match)
search_term = "go"
matching_courses = list(courses_collection.find({"title": {"$regex": search_term, "$options": "i"}}))

for course in matching_courses:
    print(f"Course: {course['title']} (ID: {course['courseId']}) - Category: {course['category']}")

Course: MongoDB (ID: EXL) - Category: Database
Course: Go (ID: GOT) - Category: Programming


Task 3.3: Update Operations

Write Python code to:

    Update a user’s profile information
    Mark a course as published
    Update assignment grades
    Add tags to an existing course




In [30]:
#update a user profile information 
user = users_collection.find_one({"firstname": "Mayowa", "lastname": "Williams"})
if user:
    updated_profile = {
        "bio": "Updated bio for Mayowa",
        "avatar": "https://new-avatar-url.com/avatar.jpg",
        "skills": ["Python", "Data Analysis", "Machine Learning"]
    }
    result = users_collection.update_one(
        {"userId": user["userId"]},
        {"$set": {"profile": updated_profile, "is_active": True}}
    )
    print(f"Updated {result.modified_count} document(s) for user {user['firstname']} {user['lastname']}")

Updated 1 document(s) for user Mayowa Williams


In [31]:
#mark a course as published
course = courses_collection.find_one({"title": "Go"})
if course:
    result = courses_collection.update_one(
        {"courseId": course["courseId"]},
        {"$set": {"isPublished": True, "updatedAt": datetime.now()}}
    )
    print(f"Marked course '{course['title']}' as published. Updated {result.modified_count} document(s).")

Marked course 'Go' as published. Updated 1 document(s).


In [32]:
#update assignments grades
assignment = assignments_collection.find_one({"assignmentId": "a_0001"})
if assignment:
    updated_grades = assignment["grades"]
    # Update grades for a specific student
    updated_grades["user_1"] = 95  # Assuming user_1 is a valid studentId

    result = assignments_collection.update_one(
        {"assignmentId": assignment["assignmentId"]},
        {"$set": {"grades": updated_grades}}
    )
    print(f"Updated grades for assignment '{assignment['assignmentId']}'. Modified {result.modified_count} document(s).")

Updated grades for assignment 'a_0001'. Modified 1 document(s).


In [33]:
#add tage to an existing course
course = courses_collection.find_one({"title": "Go"})
if course:
    updated_tags = course["tags"]
    updated_tags.append("Backend")  # Adding a new tag

    result = courses_collection.update_one(
        {"courseId": course["courseId"]},
        {"$set": {"tags": updated_tags}}
    )
    print(f"Added new tag to course '{course['title']}'. Updated {result.modified_count} document(s).")

Added new tag to course 'Go'. Updated 1 document(s).


Task 3.4: Delete Operations

Write Python code to:

    Remove a user (soft delete by setting isActive to false)
    Delete an enrollment
    Remove a lesson from a course

In [34]:
#remove a user 
user = users_collection.find_one({"is_active": True, "role": "student"})
update_time = datetime.now()
if user:
    result = users_collection.update_one(
        {"userId": user["userId"]},
        {"$set": {"is_active": False, "updatedAt": update_time}}
    )
    print(f"Removed user '{user['userId']}'. Updated {result.modified_count} document(s).")



Removed user 'user_3'. Updated 1 document(s).


In [35]:
#delete an enrollement record
enrollment = enrollments_collection.find_one()
if enrollment:
    result = enrollments_collection.delete_one({"enrollmentId": enrollment["enrollmentId"]})
    print(f"Deleted enrollment '{enrollment['enrollmentId']}'. Deleted {result.deleted_count} document(s).")
    

Deleted enrollment '0000'. Deleted 1 document(s).


In [36]:
#remove a lesson from course
lesson = lessons_collection.find_one()
course = courses_collection.find_one({"courseId": lesson["courseId"]}) 
if course:
    result = lessons_collection.delete_one({"lessonId": lesson["lessonId"]})
    print(f"Deleted lesson '{lesson['lessonId']}' in {course['title']} course. Deleted {result.deleted_count} document(s).")

else: None

Deleted lesson 'lesson_486' in MongoDB course. Deleted 1 document(s).


Task 4.1: Complex Queries

Write Python code using various PyMongo operators:

    Find courses with price between $50 and $200
    Get users who joined in the last 6 months
    Find courses that have specific tags using $in operator
    Retrieve assignments with due dates in the next week



In [37]:
#find courses with prices between $50 and $200
courses_in_price_range = list(courses_collection.find({"price": {"$gte": 50, "$lte": 200}}))
for course in courses_in_price_range:
    print(f"Course: {course['title']} (ID: {course['courseId']}) - Price: ${course['price']}")

Course: Python (ID: PYT) - Price: $191
Course: MongoDB (ID: EXL) - Price: $192
Course: JavaScript (ID: JSC) - Price: $142
Course: Data Science (ID: DSI) - Price: $106
Course: Data Engineering (ID: DEG) - Price: $69
Course: Data Visualization (ID: DVI) - Price: $132
Course: SQL (ID: SQL) - Price: $198
Course: Data Analysis (ID: DAS) - Price: $123
Course: Go (ID: GOT) - Price: $157


In [53]:
#get users who joined in the last 6 months 
time_joined = datetime.now() - timedelta(days=180)
users = list(users_collection.find({"dateJoined": {"$gte": time_joined}}))

print(f"List of users that joined in the past 6 months \n")
for user in users:
    diff_days = (datetime.now() - user['dateJoined']).days 
    
    if diff_days == 0:
        time_ago = "today"
    elif diff_days < 7:
        time_ago = f"{diff_days} day(s) ago"
    elif diff_days < 30:
        week_ago = diff_days // 7
        time_ago = f"{week_ago} week(s) ago"
    else:
        month_ago = diff_days // 30
        time_ago = f"{month_ago} month(s) ago"


    print(f"{user['firstname']} {user['lastname']} joined {time_ago}")
    

List of users that joined in the past 6 months 

Adams Barbara joined 4 week(s) ago
Johnson Pelumi joined 1 month(s) ago
Emily Alice joined 5 month(s) ago
Miller Sarah joined 3 month(s) ago
Sarah Alice joined 5 month(s) ago
Williams Thomas joined 2 month(s) ago
James Robert joined 5 month(s) ago
Mayowa Williams joined today


In [56]:
#find courses that have a specific tags using $in operator
course = list(courses_collection.find({"tags":{"$in":["Python"]}}))

for c in course:
    print(f"{c['title']}, {c['tags']}, {c['courseId']}")

Python, ['Python', 'Programming'], PYT
MongoDB, ['Python', 'Database'], EXL
Data Science, ['Python', 'Data'], DSI
Data Engineering, ['Python', 'Programming'], DEG
Data Visualization, ['Python', 'Programming'], DVI


In [75]:
#retrieve assignments with due date in the next 7 days
today = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
due_date = today + timedelta(days=7)
assignments = list(assignments_collection.find({"dueDate": {"$gt" : today , "$lt" : due_date}}))

print("Assignments due in the next 7 days:\n")
for a in assignments:
    print(f"{a['courseId']}, {a['description']} dues {a['dueDate'].date()}")

Assignments due in the next 7 days:

DEG, Complete the assignment for MongoDB dues 2025-10-08
JSC, Complete the assignment for Data Visualization dues 2025-10-06
PYT, Complete the assignment for JavaScript dues 2025-10-03
DVI, Complete the assignment for MongoDB dues 2025-10-04


Task 4.2: Aggregation Pipeline

Create aggregation pipelines using PyMongo for:

    Course Enrollment Statistics:
        Count total enrollments per course
        Calculate average course rating
        Group by course category
    Student Performance Analysis:
        Average grade per student
        Completion rate by course
        Top-performing students
    Instructor Analytics:
        Total students taught by each instructor
        Average course rating per instructor
        Revenue generated per instructor
    Advanced Analytics:
        Monthly enrollment trends
        Most popular course categories
        Student engagement metrics


In [86]:
#course enrollment statistics
#  Course Enrollment Statistics:
#         Count total enrollments per course
#    Calculate average course rating....not yet done 
#         Group by course category

enrollment_stats = enrollments_collection.aggregate([
    {"$group": 
        {"_id": "$courseId", 
         "totalEnrollments": {"$sum": 1}}
    }
])

print("Enrollment stats:\n")
for stat in enrollment_stats:
    # Look up the course details
    course = courses_collection.find_one({"courseId": stat["_id"]})
    if course:
        print(f"Category: {course['category']}")
        print(f" - Course: {course['title']} (ID: {stat['_id']})")
        print(f" -  Total Enrollments: {stat['totalEnrollments']}\n")

Enrollment stats:

Category: Programming
 - Course: JavaScript (ID: JSC)
 -  Total Enrollments: 3

Category: Data
 - Course: Data Analysis (ID: DAS)
 -  Total Enrollments: 5

Category: Database
 - Course: MongoDB (ID: EXL)
 -  Total Enrollments: 2

Category: Data
 - Course: Data Engineering (ID: DEG)
 -  Total Enrollments: 1

Category: Data
 - Course: Data Visualization (ID: DVI)
 -  Total Enrollments: 1

Category: Programming
 - Course: Python (ID: PYT)
 -  Total Enrollments: 3



In [93]:
assignments_collection = db["assignments"]

# fetch all assignments
assignments = list(assignments_collection.find({}))

for a in assignments:
    if isinstance(a.get("grades"), dict):  # only convert dicts
        new_grades = [{"studentId": k, "grade": v} for k, v in a["grades"].items()]
        
        assignments_collection.update_one(
            {"_id": a["_id"]},
            {"$set": {"grades": new_grades}}
        )
        print(f"Updated assignment {a['assignmentId']}")


Updated assignment a_0000
Updated assignment a_0001
Updated assignment a_0002
Updated assignment a_0003
Updated assignment a_0004
Updated assignment a_0005
Updated assignment a_0006
Updated assignment a_0007
Updated assignment a_0008
Updated assignment a_0009


In [ ]:
assignment = list(assignments_collection.find({}))

for a in assignment:
    
    print(a)

{'_id': ObjectId('68de2afede87aa09a1f6bad8'), 'assignmentId': 'a_0000', 'dueDate': datetime.datetime(2025, 10, 20, 8, 34, 22, 751000), 'lessonId': 'lesson_395', 'courseId': 'DAS', 'description': 'Complete the assignment for JavaScript', 'grades': [{'studentId': 'user_1', 'grade': 59}, {'studentId': 'user_2', 'grade': 86}, {'studentId': 'user_3', 'grade': 88}, {'studentId': 'user_4', 'grade': 68}, {'studentId': 'user_5', 'grade': 87}, {'studentId': 'user_6', 'grade': 63}, {'studentId': 'user_7', 'grade': 68}, {'studentId': 'user_8', 'grade': 62}, {'studentId': 'user_9', 'grade': 97}, {'studentId': 'user_10', 'grade': 93}, {'studentId': 'user_11', 'grade': 82}, {'studentId': 'user_12', 'grade': 77}, {'studentId': 'user_13', 'grade': 72}, {'studentId': 'user_14', 'grade': 88}, {'studentId': 'user_15', 'grade': 77}]}
{'_id': ObjectId('68de2afede87aa09a1f6bad9'), 'assignmentId': 'a_0001', 'dueDate': datetime.datetime(2025, 10, 8, 8, 34, 22, 751000), 'lessonId': 'lesson_448', 'courseId': 'DE

In [103]:
# Student Performance Analysis:
#         Average grade per student
#         Completion rate by course ---not yet done 
#         Top-performing students

student_analysis = assignments_collection.aggregate([
    {"$unwind": "$grades"},
    {
        "$group": 
            {   "_id" : "$grades.studentId", #grouped students by id 
                "avg_grade": {"$avg": "$grades.grade"}  #average grade
            }
     },
    {
        "$sort": {"avg_grade": -1}
     },
    {
        "$setWindowFields": {
             "sortBy":{"avg_grade": -1},
            "output": {
                "rank": {"$rank": {}}
            }
        }
     }   
])

print(f"Student analysis: \n")
for student in student_analysis:
    student_doc = users_collection.find_one({"userId": student["_id"], "role":"student"})
    if student_doc:
        print(f"{student_doc['firstname']} {student_doc['lastname']} (Id: {student_doc['userId']}): ")
        print(f"    Average grade: {student['avg_grade']:.2f}")
        print(f"    Rank: {student['rank']} \n")

Student analysis: 

Miller Ruth (Id: user_11): 
    Average grade: 80.20
    Rank: 1 

Miller Sarah (Id: user_8): 
    Average grade: 77.90
    Rank: 2 

Adams Susan (Id: user_9): 
    Average grade: 77.50
    Rank: 3 

Adams Barbara (Id: user_1): 
    Average grade: 77.40
    Rank: 4 

Williams Thomas (Id: user_14): 
    Average grade: 76.20
    Rank: 5 

Adams Susan (Id: user_7): 
    Average grade: 75.80
    Rank: 6 

John Mary (Id: user_13): 
    Average grade: 74.40
    Rank: 7 

Jones Jennifer (Id: user_10): 
    Average grade: 73.10
    Rank: 8 

Miller Grace (Id: user_3): 
    Average grade: 72.10
    Rank: 9 

Sarah Alice (Id: user_12): 
    Average grade: 72.00
    Rank: 10 

Sarah Anderson (Id: user_2): 
    Average grade: 71.20
    Rank: 11 

Doe Elizabeth (Id: user_15): 
    Average grade: 70.90
    Rank: 12 

Linda Robert (Id: user_5): 
    Average grade: 70.60
    Rank: 13 

Johnson Pelumi (Id: user_4): 
    Average grade: 69.00
    Rank: 14 

Emily Alice (Id: user_6): 


In [109]:
#adding rating to the course_collection

students = list(users_collection.find({"role":"student"}))
courses = courses_collection.find({})


for course in courses:
    ratings = []
    for student in students:
        rate_value = random.randint(1, 5)
        rates = {
            "studentId": student["userId"],
            "rating": rate_value,
            "comment": "Great course, would recommend it" if rate_value > 3 else "Can be improved, thanks",
            "ratedAt": datetime.now()
        }
        ratings.append(rates)
    
    result = courses_collection.update_one(
        {"_id": course["_id"]},
        {"$set": {"ratings": ratings}}
    )
    print(f"Added ratings to '{course['_id']}'. Modified {result.modified_count} document(s).")


Added ratings to '68de2afcde87aa09a1f6baa8'. Modified 1 document(s).
Added ratings to '68de2afcde87aa09a1f6baa9'. Modified 1 document(s).
Added ratings to '68de2afcde87aa09a1f6baaa'. Modified 1 document(s).
Added ratings to '68de2afcde87aa09a1f6baab'. Modified 1 document(s).
Added ratings to '68de2afcde87aa09a1f6baac'. Modified 1 document(s).
Added ratings to '68de2afcde87aa09a1f6baad'. Modified 1 document(s).
Added ratings to '68de2afcde87aa09a1f6baae'. Modified 1 document(s).
Added ratings to '68de2afcde87aa09a1f6baaf'. Modified 1 document(s).
Added ratings to '68de2b02de87aa09a1f6bafb'. Modified 1 document(s).


In [116]:
# Instructor Analytics:
#     Total students taught by each instructor
#     Average course rating per instructor  
#     Revenue generated per instructor

instructor_stats = courses_collection.aggregate([
    # Join enrollments
    {
        "$lookup": {
            "from": "enrollments",
            "localField": "courseId",
            "foreignField": "courseId",
            "as": "enrolled_students"
        }
    },
    # Unwind ratings for avg_rating
    {"$unwind": {"path": "$ratings", "preserveNullAndEmptyArrays": True}},
    # Group by instructor
    {
        "$group": {
            "_id": "$instructorId",
            "unique_students": {"$addToSet": "$enrolled_students.studentId"}, # list of all enrolled student IDs
            "avg_rating": {"$avg": "$ratings.rating"},
            "revenue": {"$sum": {"$multiply": ["$price", {"$size": "$enrolled_students"}]}}
        }
    },
    # Count unique students
    {
        "$project": {
            "total_students": {"$size": "$unique_students"},
            "avg_rating": 1,
            "revenue": 1
        }
    },
    {"$sort": {"total_students": -1}}
])


for instructor in instructor_stats:
    doc = users_collection.find_one({"userId": instructor["_id"], "role": "instructor"})
    if doc:
        print(f"Instructor: {doc['firstname']} {doc['lastname']} (ID: {doc['userId']})")
        print(f"  Total students taught: {instructor['total_students']}")
        print(f"  Average course rating: {instructor.get('avg_rating', 0):.2f}")
        print(f"  Revenue generated: ${instructor['revenue']:.2f}\n")


Instructor: Doe Davis (ID: user_20)
  Total students taught: 4
  Average course rating: 2.73
  Revenue generated: $15072.00

Instructor: James Robert (ID: user_18)
  Total students taught: 2
  Average course rating: 3.00
  Revenue generated: $9168.00

Instructor: Doe Thomas (ID: user_17)
  Total students taught: 1
  Average course rating: 3.00
  Revenue generated: $1104.00

Instructor: Cooper Jennifer (ID: user_16)
  Total students taught: 1
  Average course rating: 3.25
  Revenue generated: $0.00

Instructor: James Jessica (ID: user_19)
  Total students taught: 1
  Average course rating: 3.25
  Revenue generated: $9840.00



In [ ]:
# Advanced Analytics:
#     Monthly enrollment trends
#     Most popular course categories
#     Student engagement metrics


# Aggregate enrollments per month

monthly_trends = enrollments_collection.aggregate([
    {
        "$group": {
            "_id": {
                "year": {"$year": "$startDate"},
                "month": {"$month": "$startDate"}
            },
            "total_enrollments": {"$sum": 1}
        }
    },
    {"$sort": {"_id.year": 1, "_id.month": 1}}
])

print("Monthly Enrollment Trends:\n")
for trend in monthly_trends:
    year = trend["_id"]["year"]
    month = trend["_id"]["month"]
    print(f"{year}-{month:02d}: {trend['total_enrollments']} enrollments")




Monthly Enrollment Trends:

2025-10: 15 enrollments


In [ ]:
# Most popular course categories

popular_categories = enrollments_collection.aggregate([
    {
        "$lookup": {
            "from": "courses",
            "localField": "courseId",
            "foreignField": "courseId",
            "as": "course_info"
        }
    },
    {"$unwind": "$course_info"},
    {
        "$group": {
            "_id": "$course_info.category",
            "total_enrollments": {"$sum": 1}
        }
    },
    {"$sort": {"total_enrollments": -1}}
])

print("\nMost Popular Course Categories:\n")
for category in popular_categories:
    print(f"{category['_id']}: {category['total_enrollments']} enrollments")



Most Popular Course Categories:

Data: 7 enrollments
Programming: 6 enrollments
Database: 2 enrollments


In [ ]:
# Student engagement metrics
# Average courses per student

engagement = enrollments_collection.aggregate([
    {
        "$group": {
            "_id": "$studentId",
            "courses_enrolled": {"$sum": 1}
        }
    },
    {
        "$group": {
            "_id": None,
            "avg_courses_per_student": {"$avg": "$courses_enrolled"},
            "total_students": {"$sum": 1}
        }
    }
])

for e in engagement:
    print(f"\nStudent Engagement Metrics:")
    print(f"Average courses per student: {e['avg_courses_per_student']:.2f}")
    print(f"Total active students: {e['total_students']}")



Student Engagement Metrics:
Average courses per student: 1.25
Total active students: 12


Task 5.1: Index Creation

Create appropriate indexes for:

    User email lookup
    Course search by title and category
    Assignment queries by due date
    Enrollment queries by student and course




In [ ]:

pipeline = [
    {"$group": {"_id": "$email", "count": {"$sum": 1}, "docs": {"$push": "$_id"}}},
    {"$match": {"count": {"$gt": 1}}}
]

duplicates = list(users_collection.aggregate(pipeline))

for dup in duplicates:
    print(dup)


for dup in duplicates:
    # keep the first document, delete the rest
    ids_to_delete = dup["docs"][1:]
    users_collection.delete_many({"_id": {"$in": ids_to_delete}})


# index for user email
users_collection.create_index(
    [("email", 1)],  # 1 for ascending
    unique=True,
    name="idx_user_email"
)



'idx_user_email'

In [126]:
# Text index for title search + category
courses_collection.create_index(
    [("title", "text"), ("category", 1)],
    name="idx_course_title_category"
)


'idx_course_title_category'

In [ ]:
# index for assignments by due date 
assignments_collection.create_index(
    [("dueDate", 1)],
    name="idx_assignment_dueDate"
)


'idx_assignment_dueDate'

In [ ]:
# index for enrollment by student and course
enrollments_collection.create_index(
    [("studentId", 1), ("courseId", 1)],
    name="idx_enrollment_student_course"
)


'idx_enrollment_student_course'

Task 5.2: Query Optimization

    Analyze query performance using explain() method in PyMongo
    Optimize at least 3 slow queries
    Document the performance improvements using Python timing functions

In [ ]:
# Utility function: benchmark query execution time
def benchmark_query(label, query_func):
    start = time.perf_counter()
    result = query_func()
    end = time.perf_counter()
    duration = (end - start) * 1000  # ms
    print(f"{label} → {duration:.3f} ms")
    return result

In [ ]:
#User email lookup
import time

query1 = {"email": "adams.susan@eduhub.com"}
print("[BEFORE] Email lookup explain:")
print(users_collection.find_one(query1))  # Sample
print(users_collection.find(query1).explain()["queryPlanner"]["winningPlan"])

benchmark_query("User email lookup", lambda: users_collection.find_one(query1))


[BEFORE] Email lookup explain:
{'_id': ObjectId('68de2afade87aa09a1f6ba9a'), 'userId': 'user_7', 'firstname': 'Adams', 'lastname': 'Susan', 'role': 'student', 'dateJoined': datetime.datetime(2024, 9, 14, 8, 34, 18, 834000), 'profile': {'bio': 'Middle foreign authority early standard. Player artist fly determine last leg. Despite heart instead. Imagine blue school.', 'avatar': 'https://placekitten.com/474/446', 'skills': ['analysis', 'python']}, 'is_active': True, 'email': 'adams.susan@eduhub.com'}
{'isCached': False, 'stage': 'EXPRESS_IXSCAN', 'keyPattern': '{ email: 1 }', 'indexName': 'idx_user_email'}
User email lookup → 1.717 ms


{'_id': ObjectId('68de2afade87aa09a1f6ba9a'),
 'userId': 'user_7',
 'firstname': 'Adams',
 'lastname': 'Susan',
 'role': 'student',
 'dateJoined': datetime.datetime(2024, 9, 14, 8, 34, 18, 834000),
 'profile': {'bio': 'Middle foreign authority early standard. Player artist fly determine last leg. Despite heart instead. Imagine blue school.',
  'avatar': 'https://placekitten.com/474/446',
  'skills': ['analysis', 'python']},
 'is_active': True,
 'email': 'adams.susan@eduhub.com'}

In [133]:
#Assignments due in the next 7 days
due_query = {
    "dueDate": {
        "$gte": datetime.now(),
        "$lte": datetime.now() + timedelta(days=7)
    }
}
print("\n[BEFORE] Assignments due explain:")
print(assignments_collection.find(due_query).explain()["queryPlanner"]["winningPlan"])

benchmark_query("Assignments due in 7 days", lambda: list(assignments_collection.find(due_query)))



[BEFORE] Assignments due explain:
{'isCached': False, 'stage': 'FETCH', 'inputStage': {'stage': 'IXSCAN', 'keyPattern': {'dueDate': 1}, 'indexName': 'idx_assignment_dueDate', 'isMultiKey': False, 'multiKeyPaths': {'dueDate': []}, 'isUnique': False, 'isSparse': False, 'isPartial': False, 'indexVersion': 2, 'direction': 'forward', 'indexBounds': {'dueDate': ['[new Date(1759418424717), new Date(1760023224717)]']}}}
Assignments due in 7 days → 1.367 ms


[{'_id': ObjectId('68de2afede87aa09a1f6badc'),
  'assignmentId': 'a_0004',
  'dueDate': datetime.datetime(2025, 10, 3, 8, 34, 22, 751000),
  'lessonId': 'lesson_335',
  'courseId': 'PYT',
  'description': 'Complete the assignment for JavaScript',
  'grades': [{'studentId': 'user_1', 'grade': 79},
   {'studentId': 'user_2', 'grade': 80},
   {'studentId': 'user_3', 'grade': 51},
   {'studentId': 'user_4', 'grade': 65},
   {'studentId': 'user_5', 'grade': 70},
   {'studentId': 'user_6', 'grade': 85},
   {'studentId': 'user_7', 'grade': 90},
   {'studentId': 'user_8', 'grade': 69},
   {'studentId': 'user_9', 'grade': 60},
   {'studentId': 'user_10', 'grade': 89},
   {'studentId': 'user_11', 'grade': 85},
   {'studentId': 'user_12', 'grade': 72},
   {'studentId': 'user_13', 'grade': 67},
   {'studentId': 'user_14', 'grade': 84},
   {'studentId': 'user_15', 'grade': 78}]},
 {'_id': ObjectId('68de2afede87aa09a1f6badf'),
  'assignmentId': 'a_0007',
  'dueDate': datetime.datetime(2025, 10, 4, 8

In [134]:
#Enrollments by student and course
enrollment_query = {"studentId": "user_1", "courseId": "CSE101"}
print("\n[BEFORE] Enrollment query explain:")
print(enrollments_collection.find(enrollment_query).explain()["queryPlanner"]["winningPlan"])

benchmark_query("Enrollment lookup", lambda: list(enrollments_collection.find(enrollment_query)))



[BEFORE] Enrollment query explain:
{'isCached': False, 'stage': 'FETCH', 'inputStage': {'stage': 'IXSCAN', 'keyPattern': {'studentId': 1, 'courseId': 1}, 'indexName': 'idx_enrollment_student_course', 'isMultiKey': False, 'multiKeyPaths': {'studentId': [], 'courseId': []}, 'isUnique': False, 'isSparse': False, 'isPartial': False, 'indexVersion': 2, 'direction': 'forward', 'indexBounds': {'studentId': ['["user_1", "user_1"]'], 'courseId': ['["CSE101", "CSE101"]']}}}
Enrollment lookup → 4.081 ms


[]

In [135]:
#Step 3: Create Optimized Indexes
# 1. Users - unique email lookup
users_collection.create_index([("email", 1)], unique=True, name="idx_user_email")

# 2. Assignments - query by due date
assignments_collection.create_index([("dueDate", 1)], name="idx_assignment_dueDate")

# 3. Enrollments - compound index for student & course
enrollments_collection.create_index([("studentId", 1), ("courseId", 1)], name="idx_enrollment_student_course")


'idx_enrollment_student_course'

In [136]:
#Benchmark After Optimization
# User email lookup
benchmark_query("User email lookup (AFTER)", lambda: users_collection.find_one(query1))

# Assignments due query
benchmark_query("Assignments due in 7 days (AFTER)", lambda: list(assignments_collection.find(due_query)))

# Enrollment query
benchmark_query("Enrollment lookup (AFTER)", lambda: list(enrollments_collection.find(enrollment_query)))


User email lookup (AFTER) → 3.006 ms
Assignments due in 7 days (AFTER) → 1.805 ms
Enrollment lookup (AFTER) → 1.244 ms


[]

Task 6.1: Schema Validation

Implement validation rules for:

    Required fields
    Data type validation
    Enum value restrictions
    Email format validation



In [138]:
#users Collection Validation
db.command({
    "collMod": "users",
    "validator": {
        "$jsonSchema": {
            "bsonType": "object",
            "required": ["firstname", "lastname", "email", "role", "password"],
            "properties": {
                "firstname": {"bsonType": "string"},
                "lastname": {"bsonType": "string"},
                "email": {
                    "bsonType": "string",
                    "pattern": "^[^@\\s]+@[^@\\s]+\\.[^@\\s]+$",  # Email format validation
                    "description": "Must be a valid email address"
                },
                "role": {
                    "enum": ["student", "instructor", "admin"],  # Enum restriction
                    "description": "Role must be one of: student, instructor, admin"
                },
                "password": {"bsonType": "string"}
            }
        }
    },
    "validationLevel": "strict"
})

{'ok': 1.0}

In [139]:
# courses Collection Validation
db.command({
    "collMod": "courses",
    "validator": {
        "$jsonSchema": {
            "bsonType": "object",
            "required": ["title", "category", "instructorId"],
            "properties": {
                "title": {"bsonType": "string"},
                "category": {
                    "enum": ["Science", "Technology", "Engineering", "Mathematics", "Arts"],
                    "description": "Category must be one of predefined values"
                },
                "instructorId": {"bsonType": "string"}
            }
        }
    },
    "validationLevel": "strict"
})


{'ok': 1.0}

In [140]:
#assignment collection validation 
db.command({
    "collMod": "assignments",
    "validator": {
        "$jsonSchema": {
            "bsonType": "object",
            "required": ["title", "courseId", "dueDate"],
            "properties": {
                "title": {"bsonType": "string"},
                "courseId": {"bsonType": "string"},
                "dueDate": {"bsonType": "date"}
            }
        }
    },
    "validationLevel": "strict"
})


{'ok': 1.0}

In [141]:
#enrollments collection validation
db.command({
    "collMod": "enrollments",
    "validator": {
        "$jsonSchema": {
            "bsonType": "object",
            "required": ["studentId", "courseId", "enrolledAt"],
            "properties": {
                "studentId": {"bsonType": "string"},
                "courseId": {"bsonType": "string"},
                "enrolledAt": {"bsonType": "date"}
            }
        }
    },
    "validationLevel": "strict"
})


{'ok': 1.0}

Task 6.2: Error Handling

Write queries that handle common errors:

    Duplicate key errors
    Invalid data type insertions
    Missing required fields


Bonus Challenges (Extra Credit)

    Create a recommendation system using aggregation
    Design a data archiving strategy for old enrollments
   


In [12]:
#create a recommendation system using aggregation framework

enrollments_collection = db["enrollments"]

def recommend_for_student(student_id, limit=10):
    pipeline = [
        # 1. Find courses this student is enrolled in
        {"$match": {"studentId": student_id}},
        {"$group": {"_id": None, "enrolledCourseIds": {"$addToSet": "$courseId"}}},
        
        # 2. Lookup course docs to get tags and categories
        {"$lookup": {
            "from": "courses",
            "localField": "enrolledCourseIds",
            "foreignField": "courseId",
            "as": "enrolledCourses"
        }},
        {"$project": {
            "enrolledCourseIds": 1,
            "enrolledTags": {"$reduce": {
                "input": "$enrolledCourses.tags",
                "initialValue": [],
                "in": {"$setUnion": ["$$value", "$$this"]}
            }},
            "enrolledCategories": {"$map": {"input": "$enrolledCourses","as":"c","in":"$$c.category"}},
            "enrolledCourseIds": 1
        }},

        # 3. Find candidate courses (exclude already enrolled)
        {"$lookup": {
            "from": "courses",
            "pipeline": [
                {"$match": {}},
                {"$project": {"courseId": 1, "title":1, "tags":1, "category":1, "price":1}}
            ],
            "as": "allCourses"
        }},
        {"$unwind": "$allCourses"},

        # 4. Filter out already enrolled courses
        {"$match": {"$expr": {"$not": {"$in": ["$allCourses.courseId", "$enrolledCourseIds"]}}}},

        # 5. Score candidates:
        #    tagOverlap = size(intersection(enrolledTags, candidate.tags))
        #    categoryBonus = candidate.category in enrolledCategories ? 1 : 0
        #    popularityBonus = enrollments count for candidate (we'll join the enrollments_collection)
        {"$lookup": {
            "from": "enrollments",
            "localField": "allCourses.courseId",
            "foreignField": "courseId",
            "as": "candidate_enrollments"
        }},
        {"$addFields": {
            "tagOverlap": {"$size": {"$setIntersection": ["$enrolledTags", "$allCourses.tags"]}},
            "categoryMatch": {"$cond": [{"$in": ["$allCourses.category", "$enrolledCategories"]}, 1, 0]},
            "popularity": {"$size": "$candidate_enrollments"},
            "candidateCourse": "$allCourses"
        }},
        
        # 6. Compute a combined score (weights can be tuned)
        {"$addFields": {
            "score": {
                "$add": [
                    {"$multiply": ["$tagOverlap", 3]},       # strong weight for tag overlap
                    {"$multiply": ["$categoryMatch", 2]},    # moderate for same category
                    {"$multiply": ["$popularity", 0.5]}      # smaller weight for popularity
                ]
            }
        }},

        # 7. Group back to list and sort by score
        {"$replaceRoot": {"newRoot": "$candidateCourse"}},
        {"$lookup": {"from":"enrollments","localField":"courseId","foreignField":"courseId","as":"_enr"}},
        {"$addFields": {"popularity": {"$size":"$_enr"}}},
        {"$sort": {"score": -1, "popularity": -1}},
        {"$limit": limit}
    ]

    # Start pipeline with a "virtual" enrollments collection for the student
    # First stage expects the enrollments for that student, so we run pipeline by injecting initial pipeline:
    student_enrollments = list(enrollments_collection.find({"studentId": student_id}, {"courseId":1}))
    if not student_enrollments:
        # fallback: recommend globally popular courses
        return list(courses_collection.aggregate([
            {"$lookup":{"from":"enrollments","localField":"courseId","foreignField":"courseId","as":"_enr"}},
            {"$addFields":{"popularity":{"$size":"$_enr"}}},
            {"$sort":{"popularity": -1}},
            {"$limit": limit}
        ]))
    # run main pipeline with a $facet-like approach by starting pipeline with a $documents stage:
    docs = [{"studentId": student_id, "courseId": e["courseId"]} for e in student_enrollments]
    # use aggregate with $documents (MongoDB 4.4+). If $documents not available, fallback to a smaller custom approach.
    return list(enrollments_collection.aggregate([{"$replaceRoot": {"newRoot": {"enrollments": docs}}}] + pipeline))

# Example usage:
recs = recommend_for_student("user_1", limit=5)
print("\nRecommended courses for user_1:\n")
for c in recs:
    print(c["courseId"], c["title"], c.get("popularity"))



Recommended courses for user_1:

DAS Data Analysis 5
PYT Python 3
JSC JavaScript 3
EXL MongoDB 2
DVI Data Visualization 1


In [14]:
# Design a data archiving strategy for old enrollments

from datetime import datetime, timedelta
from pymongo import WriteConcern
from pymongo.errors import BulkWriteError

RETENTION_YEARS = 2
batch_size = 1000

cutoff = datetime.now() - timedelta(days=RETENTION_YEARS * 365)
query = {"startDate": {"$lte": cutoff}}  # or "endDate" depending on your data

def archive_old_enrollments():
    total_archived = 0
    while True:
        docs = list(enrollments_collection.find(query).limit(batch_size))
        if not docs:
            break
        # prepare docs for archive
        for d in docs:
            d["_archivedAt"] = datetime.now()
        # insert into archive collection
        try:
            enrollments_archive_collection.insert_many(docs, ordered=False)
        except BulkWriteError as bwe:
            # handle duplicates or errors
            print("BulkWriteError:", bwe.details)
        # delete from original
        ids = [d["_id"] for d in docs]
        res = enrollments_collection.delete_many({"_id": {"$in": ids}})
        total_archived += res.deleted_count
        print(f"Archived batch of {res.deleted_count} enrollments")
    print(f"Total archived: {total_archived}")

# run the job
archive_old_enrollments()


Total archived: 0


In [8]:
#adding resource field to the lessons_collection

from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["EduHub"]
lessons_collection = db["lessons"]

# Update one lesson by ID
lessons_collection.update_many(
    {},   # empty filter → all documents
    {"$set": {
        "resources": [
            { "type": "video", "url": "https://example.com/default_video.mp4" },
            { "type": "pdf", "url": "https://example.com/default_doc.pdf" }
        ]
    }}
)



UpdateResult({'n': 25, 'nModified': 25, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [9]:
#adding fileUrl field to the submissions_collection for submission link. 

from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["EduHub"]
submissions_collection = db["submissions"]

# Update one lesson by ID
submissions_collection.update_many(
    {},   # empty filter → all documents
    {"$set": {
        "fileUrl": "https://example.com/uploads/user1_project.zip",
    }}
)


UpdateResult({'n': 12, 'nModified': 12, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)